# Introduction #

Run the cell below to set everything up.

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.computer_vision.ex6 import *

# Imports
import visiontools
from visiontools import StanfordCars
import tensorflow as tf
import tensorflow_datasets as tfds

# Load training and validation sets
DATA_DIR = '/kaggle/input/stanford-cars-for-learn/'
(ds_train_, ds_valid_), ds_info = tfds.load(
    'stanford_cars/simple_2',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
    data_dir=DATA_DIR,
    download=False,
)

# Create data pipeline
BATCH_SIZE = 32
AUTO = tf.data.experimental.AUTOTUNE
SIZE = [192, 192]

preprocess = visiontools.make_preprocessor(SIZE)

ds_train = (ds_train_
            .map(preprocess, AUTO)
            .cache()
#            .shuffle(1000)
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

ds_valid = (ds_valid_
            .map(preprocess, AUTO)
            .cache()
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

Run the cell below if you'd like to see a few examples of the new dataset.

In [ ]:
tfds.show_examples(ds_train_, ds_info)

# Fine Tune a Model

This time you'll use the custom convnet you made in Lesson 2 as the base model.

### 1) Define Model

Here is a diagram for the model you'll define:

<!--TODO: ex6.1 model-->

In [ ]:
import tensorflow.keras.layers as layers
from tensorflow.keras import Sequential

NUM_CLASSES = ds_info.features['label'].num_classes

pretrained_base = tf.keras.models.load_model('mini_vgg_headless.h5')
pretrained_base.trainable = False

model = Sequential([
    # YOUR CODE HERE
    ----
])
q_1.check()

You can see the model you defined by running the following cell, if you'd like to compare.

In [ ]:
model.summary()

In [ ]:
import tensorflow.keras.layers as layers
from tensorflow.keras import Sequential

NUM_CLASSES = ds_info.features['label'].num_classes

pretrained_base = tf.keras.models.load_model('mini_vgg_headless.h5')
pretrained_base.trainable = False

model = Sequential([
    pretrained_base,
    layers.Flatten(),
    layers.Dense(4, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

### 2) Train Head

Compile the model with the following parameters:
- 10 epochs
- `'binary_crossentropy'` loss
- `'AUC'` metric

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.002,
    decay_steps=41,
    decay_rate=0.9,
    staircase=True,
)
optimizer = tf.keras.optimizers.Adam(lr_schedule)

# Number of epochs per round of training
# YOUR CODE HERE
EPOCHS = ____

model.compile(
    optimizer=optimizer,
    # YOUR CODE HERE
    ____
)
q_2.check()

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.002,
    decay_steps=41,
    decay_rate=0.9,
    staircase=True,
)
optimizer = tf.keras.optimizers.Adam(lr_schedule)

# Number of epochs per round of training
EPOCHS = 5

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['AUC'],
)

Once you've got the right answer, run the cell below to train your model.

In [ ]:
history = model.fit(ds_train,
                    validation_data=ds_valid,
                    epochs=EPOCHS)

### 3) Fine Tune Base

Let's fine tune the convolutional layers in the last block of the model. Run the cell below to get a list of their indices.

In [ ]:
for idx, layer in enumerate(pretrained_base.layers):
    print(idx, layer.name)

What are the indices you'll select for retraining?


In [ ]:
# YOUR CODE HERE
idx_tuned=[____]

q_3.check()

Once you've got the right answer, run the cell below to start the training loop.

In [ ]:
for r, idx in enumerate(idx_tuned):
    print("Unfreezing layer at index {}.".format(idx))
    pretrained_base.layers[idx].trainable = True

    # Recompile model after unfreezing a layer.
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['AUC'],
    )
    # Define epochs
    INIT, = history.epoch[-1] + 1, # start after last iteration's end
    TOTAL = history.epoch[-1] + 1 + EPOCHS
    # Retrain with layer at idx unfrozen.
    history = model.fit(
        ds_train,
        validation_data=ds_valid,
        initial_epoch=INIT,
        epochs=TOTAL,
    )
    # Concatenate this round's history to previous history
    history_frame = pd.concat([history_frame, pd.DataFrame(history.history)])

### 4) Evaluate

Run the cell below to see the training curves.

In [ ]:
import pandas as pd

history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['AUC', 'val_AUC']].plot();

What do you think?

In [ ]:
q_4.solution()

# Conclusion #

The technique we saw in this lesson is just one way to do transfer learning. There are several others which might be better in other situations.

- save "bottlenecks"
- frozen base (like in Lesson 1)
- fine tuning entire base with warmup

We encourage you to explore these on your own!

The technique we'll look at in the next lesson -- **data augmentation** -- is a great complement to fine tuning.